# Benchmark of Matrix Multiplications
On this benchmark we compare several operations using numpy, numexpr and numba (CPU&GPU).

In [1]:
import sys
import os
import numpy as np
import numexpr as ne
from numba import vectorize
import math
from functools import reduce
import pandas as pd
import bokeh
from utils import (get_number_processors, get_ram_memory, get_total_gpu_memory, 
                   get_gpu_name, get_cuda_version, get_cudnn_version, AttributeDict)

print("System version: {}".format(sys.version))
print("Numpy version: {}".format(np.__version__))
print("Pandas version: {}".format(pd.__version__))
print("Numexpr version: {}".format(ne.__version__))


%load_ext autoreload
%autoreload 2

System version: 3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]
Numpy version: 1.16.0
Pandas version: 0.23.4
Numexpr version: 2.6.9


## Helper function with numpy

In [2]:
def multiply(a,b):
    return a*b

def exponential(a, b):
    return a*np.exp(b)

def sine(a, b):
    return a*np.sin(b)

# A general function that multiplies an arbitrary number of matrices
# is 28% slower than directly multiplying the factors.
# The function multiply_list is not used, just leaving it here for reference
def multiply_list(l):
    return reduce(lambda x, y: x*y, l) 

def multiply3(a, b, c):
    return a*b*c

def multiply5(a, b, c, d, e):
    return a*b*c*d*e

def exponential_sine(a, b, c):
    return a*np.exp(b)*np.sin(c)

## Helper function with numpexp

In [3]:
def ne_multiply(a,b):
    return ne.evaluate("a*b")

def ne_exponential(a, b):
    return ne.evaluate("a*exp(b)")

def ne_sine(a, b):
    return ne.evaluate("a*sin(b)")

def ne_multiply3(a, b, c):
    return ne.evaluate("a*b*c")

def ne_multiply5(a, b, c, d, e):
    return ne.evaluate("a*b*c*d*e")

def ne_exponential_sine(a, b, c):
    return ne.evaluate("a*exp(b)*sin(c)")


## Helper functions for numba
NOTE: For numba solutions, having a solution empty vector speeds up around 10%
```
r0 = np.empty((S1, S2), dtype=np.int16)
r0 = multicpu(a, b)
```
source: https://devblogs.nvidia.com/numba-python-cuda-acceleration/

In [4]:
@vectorize(["int16(int16, int16)"], target="cpu")
def multicpu(a, b):
    return a * b

@vectorize(["int16(int16, int16)"], target="cuda")
def multicuda(a, b):
    return a * b

@vectorize(["float32(float32, float32)"], target="cpu")
def multfcpu(a, b):
    return a * b

@vectorize(["float32(float32, float32)"], target="cuda")
def multfcuda(a, b):
    return a * b

@vectorize(["float32(float32, float32)"], target="cpu")
def expcpu(a, b):
    return a*math.exp(b)

@vectorize(["float32(float32, float32)"], target="cuda")
def expcuda(a, b):
    return a*math.exp(b)

@vectorize(["float32(float32, float32)"], target="cpu")
def sincpu(a, b):
    return a*math.sin(b)

@vectorize(["float32(float32, float32)"], target="cuda")
def sincuda(a, b):
    return a*math.sin(b)

@vectorize(["float32(float32, float32, float32)"], target="cpu")
def multfcpu3(a, b, c):
    return a * b * c

@vectorize(["float32(float32, float32, float32)"], target="cuda")
def multfcuda3(a, b, c):
    return a * b * c

@vectorize(["float32(float32, float32, float32, float32, float32)"], target="cpu")
def multfcpu5(a, b, c, d, e):
    return a * b * c * d * e

@vectorize(["float32(float32, float32, float32, float32, float32)"], target="cuda")
def multfcuda5(a, b, c, d, e):
    return a * b * c * d * e

@vectorize(["float32(float32, float32, float32)"], target="cpu")
def expsincpu(a, b, c):
    return a*math.exp(b)*math.sin(c)

@vectorize(["float32(float32, float32, float32)"], target="cuda")
def expsincuda(a, b, c):
    return a*math.exp(b)*math.sin(c)

## Data

In [5]:
def factors_int(s1=100, s2=100):
    a = np.random.randint(1, 5, (s1, s2), dtype=np.int16)
    b = np.random.randint(1, 10, (s1, s2), dtype=np.int16)
    return a, b

def factors_float(s1=100, s2=100):
    a = np.random.randn(s1, s2).astype(np.float32)
    b = np.random.randn(s1, s2).astype(np.float32)
    return a, b

def factors_float3(s1=100, s2=100):
    a = np.random.randn(s1, s2).astype(np.float32)
    b = np.random.randn(s1, s2).astype(np.float32)
    c = np.random.uniform(low=0, high=10, size=(s1,s2)).astype(np.float32)
    return a, b, c

def factors_float5(s1=100, s2=100):
    a = np.random.randn(s1, s2).astype(np.float32)
    b = np.random.randn(s1, s2).astype(np.float32)
    c = np.random.uniform(low=0, high=10, size=(s1,s2)).astype(np.float32)
    d = np.random.uniform(low=5, high=15, size=(s1,s2)).astype(np.float32)
    e = np.random.uniform(low=0, high=30, size=(s1,s2)).astype(np.float32)
    return a, b, c, d, e

## Benchmark

In [6]:
size_combinations=[
    (100, 100),
    (1000, 1000),
    (10000, 10000),
    (100000, 10000),
    (100000, 100000)
]

In [7]:
columns = ["n_processors",
           "cpu_memory",
           "gpu_name",
           "gpu_memory",
           "data_type",
           "size1",
           "size2",
           "operation",
           "numpy",
           "numexpr",
           "numba_cpu",
           "numba_gpu"]

In [8]:
n_processors = get_number_processors()
cpu_memory = get_ram_memory(units="Gb")
gpu_name = get_gpu_name()[0]
gpu_memory = get_total_gpu_memory(units="Gb")[0]
header = [n_processors, cpu_memory, gpu_name, gpu_memory]

In [9]:
filebase = gpu_name.replace(" ", "-")
filebase

'Tesla-V100-PCIE-16GB'

In [10]:
folder = "data"
os.makedirs(folder, exist_ok=True)

#### Integer matrix multiplication

In [11]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b = factors_int(s1, s2)
    operation = "a*b"
    r1 = %timeit -o multiply(a,b)
    r2 = %timeit -o ne_multiply(a,b)
    r3 = %timeit -o multicpu(a,b)
    try:
        r4 = %timeit -o multicuda(a,b)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))
    print("")
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row


2.1 µs ± 20.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
464 µs ± 18.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.89 µs ± 2.34 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.43 ms ± 4.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
213 µs ± 813 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
573 µs ± 8.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
214 µs ± 873 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.62 ms ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
89.5 ms ± 591 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.4 ms ± 420 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
88 ms ± 845 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
145 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
900 ms ± 5.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
157 ms ± 7.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
927

In [12]:
filename = filebase + "_" + operation + "_int" + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

,n_processors,cpu_memory,gpu_name,gpu_memory,data_type,size1,size2,operation,numpy,numexpr,numba_cpu,numba_gpu
0,24,440.909752,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.int16'>,100,100,a*b,0.000002,0.000464,0.000002,0.00143465
1,24,440.909752,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.int16'>,1000,1000,a*b,0.000213,0.000573,0.000214,0.00461905
2,24,440.909752,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.int16'>,10000,10000,a*b,0.089450,0.021387,0.088043,0.145048
3,24,440.909752,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.int16'>,100000,10000,a*b,0.899621,0.157267,0.927014,1.43426
4,24,440.909752,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.int16'>,100000,100000,a*b,8.922588,1.671576,8.855293,OOM


#### Float matrix multiplication

In [11]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b = factors_float(s1, s2)
    operation = "a*b"
    r1 = %timeit -o multiply(a,b)
    r2 = %timeit -o ne_multiply(a,b)
    r3 = %timeit -o multfcpu(a,b)
    try:
        r4 = %timeit -o multfcuda(a,b)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))
    print("")
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

2.87 µs ± 3.39 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
470 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.06 µs ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.43 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
399 µs ± 730 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
577 µs ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
423 µs ± 713 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.11 ms ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
177 ms ± 914 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.8 ms ± 455 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
176 ms ± 822 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
279 ms ± 36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.74 s ± 6.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
292 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.73 s ± 5.57 

In [12]:
filename = filebase + "_" + operation + "_float" + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

,n_processors,cpu_memory,gpu_name,gpu_memory,data_type,size1,size2,operation,numpy,numexpr,numba_cpu,numba_gpu
0,24,440.909756,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.float32'>,100,100,a*b,0.000003,0.000470,0.000003,0.0014349
1,24,440.909756,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.float32'>,1000,1000,a*b,0.000399,0.000577,0.000423,0.00511176
2,24,440.909756,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.float32'>,10000,10000,a*b,0.177347,0.031833,0.175587,0.279195
3,24,440.909756,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.float32'>,100000,10000,a*b,1.735279,0.291996,1.732133,2.81469
4,24,440.909756,Tesla V100-PCIE-16GB,15.781738,<class 'numpy.float32'>,100000,100000,a*b,17.586249,2.701125,17.506866,OOM


#### Exponential matrix multiplication

In [ ]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b = factors_float(s1, s2)
    operation = "a*exp(b)"
    r1 = %timeit -o multiply(a,b)
    r2 = %timeit -o ne_multiply(a,b)
    r3 = %timeit -o multfcpu(a,b)
    try:
        r4 = %timeit -o multfcuda(a,b)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2)) 
    print("")
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

2.87 µs ± 5.14 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
451 µs ± 34.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.27 µs ± 5.01 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.43 ms ± 5.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

398 µs ± 3.68 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
576 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
422 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.05 ms ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

176 ms ± 678 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
32 ms ± 395 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
175 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
281 ms ± 38.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

1.74 s ± 7.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
290 ms ± 7.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.73

In [ ]:
filename = filebase + "_" + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Sine matrix multiplication

In [ ]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b = factors_float(s1, s2)
    operation = "a*sin(b)"
    r1 = %timeit -o multiply(a,b)
    r2 = %timeit -o ne_multiply(a,b)
    r3 = %timeit -o multfcpu(a,b)
    try:
        r4 = %timeit -o multfcuda(a,b)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))        
    print("")
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

In [ ]:
filename = filebase + "_" + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Multiple matrix multiplication (3 factors)

In [ ]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b, c = factors_float3(s1, s2)
    operation = "a*b*c"
    r1 = %timeit -o multiply3(a,b,c)
    r2 = %timeit -o ne_multiply3(a,b,c)
    r3 = %timeit -o multfcpu3(a,b,c)
    try:
        r4 = %timeit -o multfcuda3(a,b,c)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))
    print("")
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

In [ ]:
filename = filebase + "_" + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Multiple matrix multiplication (5 factors)

In [ ]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b, c, d, e = factors_float5(s1, s2)
    operation = "a*b*c*d*e"
    r1 = %timeit -o multiply5(a,b,c,d,e)
    r2 = %timeit -o ne_multiply5(a,b,c,d,e)
    r3 = %timeit -o multfcpu5(a,b,c,d,e)
    try:
        r4 = %timeit -o multfcuda5(a,b,c,d,e)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))
    print("")
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

In [ ]:
filename = filebase + "_" + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df

#### Exponential sine matrix multiplication

In [ ]:
df = pd.DataFrame(columns=columns)
for s1, s2 in size_combinations:
    a, b, c = factors_float3(s1, s2)
    operation = "a*exp(b)*sin(c)"
    r1 = %timeit -o exponential_sine(a,b,c)
    r2 = %timeit -o ne_exponential_sine(a,b,c)
    r3 = %timeit -o expsincpu(a,b,c)
    try:
        r4 = %timeit -o expsincuda(a,b,c)
    except: # in case of Out Of Memory (OOM)
        r4 = AttributeDict()
        r4["average"] = "OOM"
        print("OOM for size ({},{})".format(s1, s2))        
    print("")
    row = header + [type(a[0,0]), s1, s2, operation, r1.average, r2.average, r3.average, r4.average]
    df.loc[len(df)] = row

In [ ]:
filename = filebase + "_" + operation + ".csv"
df.to_csv(os.path.join(folder, filename), index=False)
df